<a href="https://colab.research.google.com/github/wassima-manssour/Basic-ChatBot-NLP/blob/main/chatB_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Workflow of ChatBot**



1.   Determine the objective of the chatbot and set goals
2.   Create a conversational architecture
3. Collection of question variation
4. Select the development approach
5. Implementing the dialogie flow (NLP)
6. Testing and revising the use case



## **Import Dependencies**

In [1]:
#!pip install tflearn

In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tensorflow as tf
import tflearn 
import random
import json
import pickle

from time import sleep

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


## **Create needed functions**

In [3]:
import string

def remove_punctuation(sentence):
  sntnce_nopunt = [w for w in sentence if w not in string.punctuation ]
  return sntnce_nopunt

In [4]:
#from nltk.tokenize import word_tokenize

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)

In [5]:
def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


In [6]:
def bag_of_words(s, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for s_word in s_words:
        for i, w in enumerate(words):
            if w == s_word:
                bag[i] = 1

    return np.array(bag)

In [7]:
'''
lemmenization
 import enchant
 d = enchant.Dict("en_US")
 d.check("Hello")
 d.check("Helo")
 d.suggest("Helo")
'''

'\nlemmenization\n import enchant\n d = enchant.Dict("en_US")\n d.check("Hello")\n d.check("Helo")\n d.suggest("Helo")\n'

## **Training Data**

In [8]:
import json

with open('/content/intents.json','r') as f:
  data = json.load(f)

In [9]:
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up', 'Good morning', 'Good evening', 'hello', 'hey', "what's up"], 'responses': ['Hello!', 'Good to see you!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'bye', 'i have to go', 'gotta go'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old are you', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am old and wise', 'trying to stay fit and young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?', 'who are you?'], 'responses': ['You can call me Sally.', "I'm Sally!", "I'm Sally."], 'context_set': ''}, {'tag': 'help', 'patterns': ['Id like to ask something', 'what can you do', 'can you help me?', '

**1. Tokenize**

In [10]:
# getting informations from intents.json--
words = [] # words to be tokenized
labels = [] # tags of the intents data
x_docs = [] # initial words before processinf(tokenization, steming...)
y_docs = [] # ['word','tag']  

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = tokenize(pattern)
        words.extend(wrds)
        x_docs.append(wrds)
        y_docs.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [11]:
print(f'words: {words}\ntags: {labels} \ninitail words: {x_docs} \nword-tag: {y_docs}')

words: ['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'Good', 'morning', 'Good', 'evening', 'hello', 'hey', 'what', "'s", 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'bye', 'i', 'have', 'to', 'go', 'got', 'ta', 'go', 'how', 'old', 'how', 'old', 'are', 'you', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'who', 'are', 'you', '?', 'Id', 'like', 'to', 'ask', 'something', 'what', 'can', 'you', 'do', 'can', 'you', 'help', 'me', '?', 'can', 'i', 'tell', 'you', 'something']
tags: ['greeting', 'goodbye', 'age', 'name', 'help'] 
initail words: [['Hi'], ['How', 'are', 'you'], ['Is', 'anyone', 'there', '?'], ['Hello'], ['Good', 'day'], ['Whats', 'up'], ['Good', 'morning'], ['Good', 'evening'], ['hello'], ['hey'], ['what', "'s", 'up'], ['cya'], ['See', 'you', 'later'], ['Goodby

**2.Remove punctuation**

In [12]:
words=remove_punctuation(words)

In [13]:
print(words)

['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', 'Hello', 'Good', 'day', 'Whats', 'up', 'Good', 'morning', 'Good', 'evening', 'hello', 'hey', 'what', "'s", 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'bye', 'i', 'have', 'to', 'go', 'got', 'ta', 'go', 'how', 'old', 'how', 'old', 'are', 'you', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', 'who', 'are', 'you', 'Id', 'like', 'to', 'ask', 'something', 'what', 'can', 'you', 'do', 'can', 'you', 'help', 'me', 'can', 'i', 'tell', 'you', 'something']


**3. Steming & removing stop words**

In [21]:
nltk.download ('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
#print(stop_words)
words=(stem(word) for word in words if word not in stop_words) # lower & stem words
words = sorted(set(words)) # get just unique words
print(words)

["'s", 'ag', 'anyon', 'ask', 'bye', 'cal', 'cya', 'day', 'ev', 'go', 'good', 'goodby', 'got', 'hav', 'hello', 'help', 'hey', 'hi', 'id', 'lat', 'leav', 'lik', 'morn', 'nam', 'old', 'see', 'ta', 'tel']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
print(len(x_docs), "patterns",x_docs)
print(len(labels), "tags:", labels)
print(len(words), "unique stemmed words:", words)

32 patterns [['Hi'], ['How', 'are', 'you'], ['Is', 'anyone', 'there', '?'], ['Hello'], ['Good', 'day'], ['Whats', 'up'], ['Good', 'morning'], ['Good', 'evening'], ['hello'], ['hey'], ['what', "'s", 'up'], ['cya'], ['See', 'you', 'later'], ['Goodbye'], ['I', 'am', 'Leaving'], ['Have', 'a', 'Good', 'day'], ['bye'], ['i', 'have', 'to', 'go'], ['got', 'ta', 'go'], ['how', 'old'], ['how', 'old', 'are', 'you'], ['what', 'is', 'your', 'age'], ['how', 'old', 'are', 'you'], ['age', '?'], ['what', 'is', 'your', 'name'], ['what', 'should', 'I', 'call', 'you'], ['whats', 'your', 'name', '?'], ['who', 'are', 'you', '?'], ['Id', 'like', 'to', 'ask', 'something'], ['what', 'can', 'you', 'do'], ['can', 'you', 'help', 'me', '?'], ['can', 'i', 'tell', 'you', 'something']]
5 tags: ['greeting', 'goodbye', 'age', 'name', 'help']
28 unique stemmed words: ["'s", 'ag', 'anyon', 'ask', 'bye', 'cal', 'cya', 'day', 'ev', 'go', 'good', 'goodby', 'got', 'hav', 'hello', 'help', 'hey', 'hi', 'id', 'lat', 'leav', 'li

**4. Building the model**

In [23]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

# One hot encoding, Converting the words to numerals
for x, doc in enumerate(x_docs):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)


    output_row = out_empty[:]
    output_row[labels.index(y_docs[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [24]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 1999  | total loss: 0.40273 | time: 0.013s
| Adam | epoch: 500 | loss: 0.40273 - acc: 0.8858 -- iter: 24/32
Training Step: 2000  | total loss: 0.42368 | time: 0.017s
| Adam | epoch: 500 | loss: 0.42368 - acc: 0.8722 -- iter: 32/32
--


In [25]:
def chat():

    while True:
        inp = input("\n\nYou: ")
        if inp.lower() == 'quit':
            break

    #Porbability of correct response 
        results = model.predict([bag_of_words(inp, words)])

    # Picking the greatest number from probability
        results_index = np.argmax(results)

        tag = labels[results_index]

        for tg in data['intents']:

            if tg['tag'] == tag:
                responses = tg['responses']
                print("Bot:\t" + random.choice(responses))

In [26]:
chat()



You: hey
Bot:	Hi there, how can I help?


You: your name?
Bot:	I'm Sally!


You: ur age?
Bot:	I am old and wise


You: what can you do
Bot:	Sad to see you go :(


You: what can u do?
Bot:	Sad to see you go :(


You: can you help me?
Bot:	I can help you with whatever you tell me to.


You: ok
Bot:	Goodbye!


You: quit
